### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk

In [1]:
import json
import nltk
import numpy as np

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [22]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器


filename = "case_summary1.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    ne_list = []
    for term in main_list:
        terms_list = pynlpir.segment(term)
        for t in terms_list:
            ne_list.append(t[0])
            
    #print(ne_list)
    
    word_list = []
    
    if idx in [64,223,251,389,480,673,744,751,769,1422,1541,1853]:
        continue
    
    txt_list = pynlpir.segment(case_content)
    #print(txt_list)
    for txt in txt_list:
        if txt[0] in ne_list:
            e = (txt[0],txt[1],"S")
        else :
            e = (txt[0],txt[1],"O")
        word_list.append(e)
        
    docs.append(word_list)

print(len(docs))   

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: '

2095


In [41]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexis/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
from nltk.corpus import stopwords

for doc in docs:
    filtered = [w for w in doc if(w not in stopwords.words('chinese'))]
    print(filtered)
#data = docs


OSError: No such file or directory: '/Users/alexis/nltk_data/corpora/stopwords/chinese'

In [26]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    if postag is None:
            postag = 'none'
    
    # Common features for all words
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '-1:word=' + word1,
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        
        if postag1 is None:
            postag1 = 'none'
        
        features.extend([
            '+1:word=' + word1,
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [27]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [28]:
print(len(X_train[0][4]))
print(X_train[0][5])

10
['bias', 'word=岁', 'word.isdigit=False', 'postag=classifier', '-1:word=23', '-1:word.isdigit=True', '-1:postag=numeral', '+1:word=，', '+1:word.isdigit=False', '+1:postag=punctuation mark']


In [29]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 60918
Seconds required: 1.648

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 268139.323522
Feature norm: 1.000000
Error norm: 163335.216421
Active features: 59852
Line search trials: 1
Line search step: 0.000001
Seconds required for this iteration: 0.379

***** Iteration #2 *****
Loss: 252895.908281
Feature norm: 0.898231
Error norm: 126761.427496
Active features: 58746
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #3 *****
Loss: 245498.451016
Feature norm: 0.570453
Error norm: 199288.679123
Active features: 41898
Line search trials: 1
Line search step: 1.000000
Seconds requir

***** Iteration #42 *****
Loss: 101363.874068
Feature norm: 153.800410
Error norm: 3759.707192
Active features: 32793
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #43 *****
Loss: 100774.301738
Feature norm: 163.821704
Error norm: 4962.877582
Active features: 32600
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.201

***** Iteration #44 *****
Loss: 100341.987620
Feature norm: 170.251884
Error norm: 3459.431720
Active features: 32058
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #45 *****
Loss: 99920.355194
Feature norm: 178.360446
Error norm: 5127.367934
Active features: 32136
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #46 *****
Loss: 99610.193168
Feature norm: 183.114276
Error norm: 2757.604980
Active features: 31887
Line search trials: 1
Line search step: 

***** Iteration #83 *****
Loss: 97288.537262
Feature norm: 209.117191
Error norm: 2393.291244
Active features: 27974
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.192

***** Iteration #84 *****
Loss: 97271.194259
Feature norm: 209.287333
Error norm: 789.606261
Active features: 27958
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.194

***** Iteration #85 *****
Loss: 97257.747425
Feature norm: 209.522994
Error norm: 2795.709822
Active features: 27912
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #86 *****
Loss: 97243.490234
Feature norm: 209.676505
Error norm: 903.053487
Active features: 27898
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.197

***** Iteration #87 *****
Loss: 97233.066878
Feature norm: 209.816749
Error norm: 1594.924852
Active features: 27867
Line search trials: 1
Line search step: 1.000

***** Iteration #124 *****
Loss: 96950.416250
Feature norm: 210.355415
Error norm: 1788.193444
Active features: 27031
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #125 *****
Loss: 96946.617379
Feature norm: 210.274343
Error norm: 2411.875116
Active features: 26983
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #126 *****
Loss: 96941.050171
Feature norm: 210.242389
Error norm: 1636.638188
Active features: 26983
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.197

***** Iteration #127 *****
Loss: 96938.205632
Feature norm: 210.179588
Error norm: 2292.923007
Active features: 26972
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.194

***** Iteration #128 *****
Loss: 96932.811732
Feature norm: 210.142730
Error norm: 1251.440244
Active features: 26965
Line search trials: 1
Line search step

***** Iteration #167 *****
Loss: 96770.475952
Feature norm: 208.769017
Error norm: 2414.934029
Active features: 26479
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.203

***** Iteration #168 *****
Loss: 96764.291414
Feature norm: 208.757228
Error norm: 1506.900795
Active features: 26475
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.193

***** Iteration #169 *****
Loss: 96759.897671
Feature norm: 208.748006
Error norm: 2003.834172
Active features: 26462
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.198

***** Iteration #170 *****
Loss: 96754.610839
Feature norm: 208.737090
Error norm: 1527.879860
Active features: 26451
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.194

***** Iteration #171 *****
Loss: 96750.664401
Feature norm: 208.730300
Error norm: 2103.965682
Active features: 26437
Line search trials: 1
Line search step

In [34]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')

y_pred_train = [tagger.tag(xseq) for xseq in X_train]
y_pred_test = [tagger.tag(xseq) for xseq in X_test]

In [37]:
# Create a mapping of labels to indices
labels = {"O": 0,"S": 1}

# training data F1-score
predictions_train = np.array([labels[tag] for row in y_pred_train for tag in row])
truths_train = np.array([labels[tag] for row in y_train for tag in row])
print(classification_report(
    truths_train, predictions_train,
    target_names=["O", "B-S"]))

# test data F1-score
predictions_test = np.array([labels[tag] for row in y_pred_test for tag in row])
truths_test = np.array([labels[tag] for row in y_test for tag in row])
print(classification_report(
    truths_test, predictions_test,
    target_names=["O", "B-S"]))

             precision    recall  f1-score   support

          O       0.96      0.98      0.97    875984
        B-S       0.72      0.55      0.62     69766

avg / total       0.95      0.95      0.95    945750

             precision    recall  f1-score   support

          O       0.96      0.98      0.97    228871
        B-S       0.64      0.48      0.55     17735

avg / total       0.94      0.94      0.94    246606

